# Gibbs Reactor
Construct a simple *flowsheet* with a single unit model, which will be a Gibbs reactor and an ideal gas property package.

**Author:** Andrew Lee <andrew.lee@netl.doe.gov>

Conversion to Jupyter Notebook: Dan Gunter <dkgunter@lbl.gov>

## 1. Setup

### Load the Solver
In order to determine the output of the model, a "solver" is needed. Pyomo provides a convenient interface to choose the solver dynamically without rewriting the later code which uses it. Here we load the freely available Interior Point Optimizer (IPOPT) solver (see https://projects.coin-or.org/Ipopt). This solver can solver certain classes of non-linear problems, and like all solvers is better for some problems than others.

*Note: Installation of IPOPT should have occurred as part of the installation of the IDAES software. If this next cell prints out an error message, then you need to check that you are running this Jupyter notebook in the right environment and/or revisit your installation steps.*

In [ ]:
from pyomo.environ import SolverFactory

if SolverFactory('ipopt').available():
    solver = SolverFactory('ipopt')
    solver.options = {'tol': 1e-6,
                      'mu_init': 1e-8,
                      'bound_push': 1e-8}
    print('IPOPT solver found')
else:
    solver = None
    print('Error: IPOPT not found: Cannot solve the system!')

### Initialize the DMF
The IDAES Data Management Frameork, or DMF, is the place you can load/store models and results. For the purposes of this example its main use will be conveniently locating the documentation for objects and classes. The command below creates a new "workspace" in the current directory, where settings and data can be accessed.

In [ ]:
from idaes.dmf import magics
%dmf init . create
# Ignore warnings about "Using existing path"
# Turn down IDAES logging to minimal levels:
import logging; logging.getLogger('idaes.dmf').setLevel(logging.WARNING)

In [ ]:
from idaes.ui.report import degrees_of_freedom

## 2. Build the model
Now it is time to build our IDAES model. Underneath, this is really a Pyomo model, so some of the operations use Pyomo modules and classes.

Create a *concrete* Pyomo model, which will be initialized with data values instead of abstract symbols. You can read more about concrete vs. abstract models [here](https://software.sandia.gov/downloads/pub/coopr/CooprGettingStarted.html#_abstract_versus_concrete_models).

In [ ]:
from pyomo.environ import ConcreteModel
m = ConcreteModel()

### Initialize the flowsheet
The IDAES *flowsheet* is the top-level container for all the models and their connections. The class you use for a flowsheet is called `FlowsheetBlock`. Below, we import this class from the `core` module in the idaes package. Then we use the `%dmf help` magic to jump over to the documentation of the class for more information.

In [ ]:
from idaes.core import FlowsheetBlock
%dmf help FlowsheetBlock  # pops up a new window with the Sphinx docs

Next, we create an empty IDAES flowsheet. The `"dynamic": False` setting means that this will be a steady-state model. We add the flowsheet to the concrete model by assigning it to the `.fs` attribute of the model.

In [ ]:
m.fs = FlowsheetBlock(default={"dynamic": False})

### Add a property package to the flowsheet
The first thing we add to the flowsheet is a property package, which represents the physical properties associated with the reactor. The available property packages
are in the `idaes.property_models` subpackage. You can use the DMF help magic to look at the package and its subpackages.

In [ ]:
import idaes.property_models
%dmf help idaes.property_models

Now we will import the actual property package we need. In property packages, the main class that you will use is called `MethaneCombustionParameterBlock`. An instance of this class is assigned to the `.properties` attribute in the IDAES flowsheet.

In [ ]:
from idaes.property_models.methane_combustion_ideal import MethaneCombustionParameterBlock

m.fs.properties = MethaneCombustionParameterBlock()

### Add Gibbs reactor unit model
Once we have added our properties, we can add the model that represents a Gibbs minimization reactor. In IDAES this is found in the `idaes.unit_models.gibbs_reactor` module under the class named `GibbsReactor`. Although we have already added the properties to the flowsheet, we must remember to also tell the reactor model to use these properties. We also add a parameter that tells the model that terms for heat transfer should be constructed. 

In [ ]:
from idaes.unit_models.gibbs_reactor import GibbsReactor
m.fs.gibbs = GibbsReactor(default={"property_package": m.fs.properties,
                                   "has_heat_transfer": True})

### Admire your handiwork
You have now constructed a very simple flowsheet containing a Gibbs reactor model and associated property data. Although extremely simple, we have a complete IDAES model ready to solve. Congratulations! You can look at all the initial settings by invoking the `.display()` method on the Gibbs reactor model.

In [ ]:
m.fs.gibbs.display()

## 3. Solve the model
We will now proceed to solve this simple model in two ways:
* for a target (fixed) outlet temperature (degrees C)
* for a target (fixed) heat duty (W)

In either case, we will first set the inlet flow parameters to fixed rates, as well as set the inlet temperature. Note that all these are operations on the Gibbs reactor model, in the flowsheet attached to our concrete Pyomo model, which is notated `m.fs.gibbs`. The flow rates are set for all the inlets by using the notation `inlet`, and then selecting the component at a given point in time and adding `.fix(<VALUE>)` to set the value.

In [ ]:
m.fs.gibbs.inlet.flow_mol_comp[0, "H2"].fix(10.0)
m.fs.gibbs.inlet.flow_mol_comp[0, "N2"].fix(150.0)
m.fs.gibbs.inlet.flow_mol_comp[0, "O2"].fix(40.0)
m.fs.gibbs.inlet.flow_mol_comp[0, "CO2"].fix(1e-5)
m.fs.gibbs.inlet.flow_mol_comp[0, "CH4"].fix(30.0)
m.fs.gibbs.inlet.flow_mol_comp[0, "CO"].fix(1e-5)
m.fs.gibbs.inlet.flow_mol_comp[0, "H2O"].fix(1e-5)
m.fs.gibbs.inlet.flow_mol_comp[0, "NH3"].fix(1e-5)

Next we set the inlet temperature(s) and pressure(s) to constant values.

In [ ]:
m.fs.gibbs.inlet.temperature.fix(1500.0)
m.fs.gibbs.inlet.pressure.fix(101325.0)

### Solve for given outlet temperature
Finally, for the first type of optimization, we fix the outlet temperature.

In [ ]:
m.fs.gibbs.outlet.temperature.fix(2844.38)

We can check that we have specified all the required constraints by checking that there are zero degrees of freedom in the model. This can be done most simply by using a convenience function defined in the `idaes.ui.report` module.

In [ ]:
from idaes.ui.report import degrees_of_freedom
def check_dof(m):
    dof = degrees_of_freedom(m)
    if dof == 0:
        print('Model has zero degrees of freedom. Good to go!')
    else:
        print('Error: Model has {:d} degrees of freedom!'.format(dof))
check_dof(m)

Now that the system is set up, we simply calculate the equilibrium state with the `.initialize()` method. This invokes the Ipopt solver, which produces a fair amount of output. Since, in the case of this simple model, the equilibrium state *is* the final solution of the system, this is the only run of the solver that needs to be performed.

In [ ]:
m.fs.gibbs.initialize(outlvl=5,
                      optarg={'tol': 1e-6},
                      state_args={'temperature': 2844.38,
                                  'pressure': 101325.0,
                                  'flow_mol_comp': {'CH4': 1e-5,
                                                    'CO': 23.0,
                                                    'CO2': 7.05,
                                                    'H2': 29.0,
                                                    'H2O': 41.0,
                                                    'N2': 150.0,
                                                    'NH3': 1e-5,
                                                    'O2': 1.0}})

### Solve for a given heat duty
We can now repeat the steps above to create an identical model that we solve for a given (fixed) heat duty, instead of temperature.

In [ ]:
m.fs.gibbs.outlet[:].temperature.unfix()
m.fs.gibbs.heat_duty[:].fix(161882.303661)
check_dof(m)

In [ ]:
# instead: solver.solve(model)
m.fs.gibbs.initialize(outlvl=5,
                      optarg={'tol': 1e-6},
                      state_args={'temperature': 2844.38,
                                  'pressure': 101325.0,
                                  'flow_mol_comp': {'CH4': 1e-5,
                                                    'CO': 23.0,
                                                    'CO2': 7.05,
                                                    'H2': 29.0,
                                                    'H2O': 41.0,
                                                    'N2': 150.0,
                                                    'NH3': 1e-5,
                                                    'O2': 1.0}})

## Fin.
This is the end of this example. If you have questions or comments, please contact the IDAES team. See the [website](https://idaes.org/) for more information.